 # CSCL 2023 Automated Multi-Dimensional Analysis of Peer Feedback in Middle School Mathematics

In [ ]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
!pip uninstall tensorflow keras -y
!pip install tensorflow

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Successfully uninstalled tensorflow-2.17.1
Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Successfully uninstalled keras-3.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import random
import nltk
import os
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn import tree, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


random.seed(20)

In [ ]:
# split data for student level cross validation: one student is nested within one fold
group_dict = dict()
groups = np.array([])

df = pd.read_csv('Annotations_final.csv')

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])

# Set up the splitter with 5 splits
gkf = GroupKFold(n_splits = 5)

## bag of words

In [ ]:
# bag of words + neural nets
# import data
# stemming
porter = PorterStemmer() # stemming recovers root words from plurals etc
stemmed_texts = []

X_text = df['annotation_text']

for answer in X_text:
    answer = ' '.join(porter.stem(word) for word in answer.split(' '))
    stemmed_texts.append(answer)

X, y = np.array(stemmed_texts), df['relating_to_self'] # CHANGE y HERE

#vect = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words="english") #only unigram 313 features
vect = CountVectorizer(ngram_range=(2,2), max_features=1000, stop_words="english") #only bigram 728 features

X = vect.fit_transform(X).toarray()


# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
accuracy_scores = np.array([])

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    model = Sequential()
    model.add(Dense(12, input_shape=(738,), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['acc']
             )

    num_epochs = 30
    batch_size = 10

    model.fit(
        X_train,
        y_train,
        epochs=num_epochs,
        validation_split=0.1,
        shuffle=True,
        batch_size=batch_size)

    predictions = model.predict(X_test)

    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - acc: 0.6781 - loss: 0.6864 - val_acc: 0.8947 - val_loss: 0.6716
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8789 - loss: 0.6644 - val_acc: 0.8421 - val_loss: 0.6573
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.9119 - loss: 0.6438 - val_acc: 0.8421 - val_loss: 0.6402
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9348 - loss: 0.6163 - val_acc: 0.8421 - val_loss: 0.6189
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9554 - loss: 0.5744 - val_acc: 0.8421 - val_loss: 0.5941
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9531 - loss: 0.5460 - val_acc: 0.8947 - val_loss: 0.5659
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9599 - loss: 0.4890 - val_acc: 0.8947 - val_loss: 0.5333
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.9810 - loss: 0.4360 - val_acc: 0.8947 - val_loss: 0.4998
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9731 - loss:

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.5245 - loss: 0.6864 - val_acc: 0.7895 - val_loss: 0.6639
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7940 - loss: 0.6463 - val_acc: 0.8947 - val_loss: 0.6244
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7845 - loss: 0.6071 - val_acc: 0.8947 - val_loss: 0.5846
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8110 - loss: 0.5591 - val_acc: 0.8947 - val_loss: 0.5419
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7568 - loss: 0.5296 - val_acc: 0.8947 - val_loss: 0.4949
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8398 - loss: 0.4532 - val_acc: 0.8947 - val_loss: 0.4479
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8624 - loss: 0.4073 - val_acc: 0.9474 - val_loss: 0.3997
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8844 - loss: 0.3696 - val_acc: 0.9474 - val_loss: 0.3602
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8985 - loss: 0.3061 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.5848 - loss: 0.6893 - val_acc: 0.8421 - val_loss: 0.6645
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.7615 - loss: 0.6533 - val_acc: 0.8421 - val_loss: 0.6339
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.8885 - loss: 0.6101 - val_acc: 0.8421 - val_loss: 0.5981
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8433 - loss: 0.5674 - val_acc: 0.8421 - val_loss: 0.5585
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8330 - loss: 0.5259 - val_acc: 0.8947 - val_loss: 0.5146
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8817 - loss: 0.4511 - val_acc: 0.8947 - val_loss: 0.4659
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8856 - loss: 0.4064 - val_acc: 0.8947 - val_loss: 0.4203
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8934 - loss: 0.3468 - val_acc: 0.8947 - val_loss: 0.3785
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9387 - loss: 0.2967 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - acc: 0.3339 - loss: 0.7035 - val_acc: 0.6842 - val_loss: 0.6910
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7067 - loss: 0.6837 - val_acc: 0.8947 - val_loss: 0.6758
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8963 - loss: 0.6649 - val_acc: 0.8947 - val_loss: 0.6610
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9186 - loss: 0.6456 - val_acc: 0.8947 - val_loss: 0.6426
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8916 - loss: 0.6232 - val_acc: 0.8947 - val_loss: 0.6192
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9491 - loss: 0.5828 - val_acc: 0.8947 - val_loss: 0.5886
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9602 - loss: 0.5340 - val_acc: 0.8947 - val_loss: 0.5520
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9625 - loss: 0.4836 - val_acc: 0.8947 - val_loss: 0.5106
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9589 - loss: 0.4232 - v

## USE

In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
from getpass import getpass
import openai
import os

#os.kill(os.getpid(), 9)

print('Enter OpenAI API key:')
openai.api_key = getpass()

os.environ['OPENAI_API_KEY']=openai.api_key

Enter OpenAI API key:
··········


In [ ]:
import time

def generate_zero_shot(topic):
  prompt = """
  I will give you a peer review statement on a math problem solution, and this statement is identified to have this key attribute:
  "Commenting on the Process (CP): When evaluating peers’ works, learners comment on the process of the work". Your task is to simplify
  this statement by removing information that is not relevant to this attribute, leaving only the core description that represents the attribute of CP.
  A good example of statement having this attribute is: "I like how you had 20 then subtracted 12 and got 8".
  """
  system_message = {"role" : "system", "content" : prompt}

  client = openai.OpenAI()
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= [
          system_message,
          {"role" : "user", "content" : topic} # simulate a user prompt
      ],
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n"]
  )
  # I recommend putting a short wait after each call,
  # since the rate limit for the platform is 60 requests/min.
  # (This increases to 3000 requests/min after you've been using the platform for 2 days).
  time.sleep(1)

  # the response from OpenAI's API is a JSON object that contains
  # the completion to your prompt plus some other information.  Here's how to access
  # just the text of the completion.
  return response.choices[0].message.content.strip()

def update_comments_with_gpt(df):
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if comment_process is 1
        if row['comment_process'] == 1:
            # Extract the annotation_text
            original_text = row['annotation_text']

            # Call the generate_zero_shot function
            try:
                refined_text = generate_zero_shot(original_text)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                refined_text = original_text  # Fallback to original if GPT fails

            # Update the annotation_text in the DataFrame
            df.at[index, 'annotation_text'] = refined_text
            print(f"Row {index} updated: {original_text} -> {refined_text}")

    return df

df = pd.read_csv('Annotations_final.csv')

# Update the DataFrame
updated_df = update_comments_with_gpt(df)

# Save the updated DataFrame to a new CSV
updated_df.to_csv('updated_annotations.csv', index=False)

print("CSV updated and saved as 'updated_annotations.csv'.")

Row 15 updated: I like how you had 20 then subtected 12 and got 8 then you added 8 and got 16 then you answer was -4  -> I like how you subtracted 12 from 20 and got 8, then added 8 to get 16, but your final answer was -4.
Row 16 updated: the video is fine i like how you add the numbers  -> I like how you add the numbers.
Row 18 updated: i like how you add the number -> You did a great job adding the numbers.
Row 23 updated: I like the way you  put how much $20 he have and the lost 12 -> I like how you had 20 then subtracted 12 and got 8.
Row 27 updated: I like the way you used arrows in your problem. -> I like how you used arrows.
Row 31 updated: The way that you set up your Data is great. And how you divided to get to your mean. GOOD JOB KALLI :) -> I like how you set up your data and calculated the mean. Great job!
Row 33 updated: I like the way you sorted the data.
I also like the way you showed your work clearly so it wouldn't be confusing. -> You sorted the data nicely and presen

In [10]:
# loading universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [11]:
# import data
df = pd.read_csv("updated_annotations.csv").fillna(0)

# extract features as X
X = df[['annotation_text','created_by']]

# extract the prediction variable as y
y = df.comment_process # CHANGE y HERE

# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
pred = pd.DataFrame()

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):

    X_train = X.iloc[train_index].drop(['created_by'], axis=1)
    X_test = X.iloc[test_index].drop(['created_by'], axis=1)
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

    # train classifier on this round of training group
    training_embeddings = embed(X_train.annotation_text.to_list())

    model = Sequential()
    model.add(Dense(12, input_shape=(512,), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics = ['acc'])

    num_epochs = 30
    batch_size = 10


    model.fit(
        training_embeddings,
        y_train,
        epochs=num_epochs,
        validation_split=0.1,
        shuffle=True,
        batch_size=batch_size)

    # test classifier on this round of testing group
    testing_embeddings = embed(X_test.annotation_text.to_list())
    predictions = model.predict(testing_embeddings)

    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))


# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - acc: 0.7672 - loss: 0.6827 - val_acc: 0.7895 - val_loss: 0.6554
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.8264 - loss: 0.6457 - val_acc: 0.7895 - val_loss: 0.6083
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - acc: 0.8441 - loss: 0.5973 - val_acc: 0.7895 - val_loss: 0.5571
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - acc: 0.8704 - loss: 0.5389 - val_acc: 0.7895 - val_loss: 0.5033
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.8907 - loss: 0.4742 - val_acc: 0.7895 - val_loss: 0.4468
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.8795 - loss: 0.4426 - val_acc: 0.8947 - val_loss: 0.3980
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.9164 - loss: 0.3718 - val_acc: 0.8947 - val_loss: 0.3568
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9228 - loss: 0.3723 - val_acc: 0.8947 - val_loss: 0.3186
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9266 - loss: 0.3338

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.6869 - loss: 0.6863 - val_acc: 0.7368 - val_loss: 0.6591
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7116 - loss: 0.6530 - val_acc: 0.7368 - val_loss: 0.6143
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.7331 - loss: 0.5942 - val_acc: 0.7368 - val_loss: 0.5563
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.7667 - loss: 0.5336 - val_acc: 0.7368 - val_loss: 0.4934
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8021 - loss: 0.4818 - val_acc: 0.7895 - val_loss: 0.4314
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8547 - loss: 0.4324 - val_acc: 0.8947 - val_loss: 0.3705
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.8793 - loss: 0.3869 - val_acc: 0.9474 - val_loss: 0.3214
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9159 - loss: 0.3110 - val_acc: 0.9474 - val_loss: 0.2744
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9346 - loss: 0.2802 - v

...

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - acc: 0.6615 - loss: 0.6808 - val_acc: 0.5789 - val_loss: 0.6742
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7480 - loss: 0.6103 - val_acc: 0.5789 - val_loss: 0.6379
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.6756 - loss: 0.5769 - val_acc: 0.5789 - val_loss: 0.5882
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7107 - loss: 0.5103 - val_acc: 0.5789 - val_loss: 0.5422
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.7641 - loss: 0.4623 - val_acc: 0.6842 - val_loss: 0.4827
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8541 - loss: 0.4083 - val_acc: 0.8421 - val_loss: 0.4286
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8692 - loss: 0.3623 - val_acc: 0.8421 - val_loss: 0.3839
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9210 - loss: 0.3328 - val_acc: 0.8421 - val_loss: 0.3326
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9449 - loss: 0.2639 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - acc: 0.6005 - loss: 0.6856 - val_acc: 0.6316 - val_loss: 0.6732
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7720 - loss: 0.6386 - val_acc: 0.6316 - val_loss: 0.6390
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - acc: 0.7723 - loss: 0.5700 - val_acc: 0.6316 - val_loss: 0.5980
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.8009 - loss: 0.4838 - val_acc: 0.6316 - val_loss: 0.5519
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7741 - loss: 0.4632 - val_acc: 0.6842 - val_loss: 0.4708
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9051 - loss: 0.3536 - val_acc: 0.8421 - val_loss: 0.3926
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9401 - loss: 0.2968 - val_acc: 0.8421 - val_loss: 0.3532
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9144 - loss: 0.3012 - val_acc: 0.8947 - val_loss: 0.3029
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9589 - loss: 0.2271 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - acc: 0.4193 - loss: 0.6954 - val_acc: 0.6842 - val_loss: 0.6862
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.7327 - loss: 0.6805 - val_acc: 0.6316 - val_loss: 0.6729
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7113 - loss: 0.6601 - val_acc: 0.6316 - val_loss: 0.6484
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.7861 - loss: 0.6303 - val_acc: 0.7895 - val_loss: 0.6042
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.8197 - loss: 0.5825 - val_acc: 0.7895 - val_loss: 0.5506
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8887 - loss: 0.5308 - val_acc: 0.8421 - val_loss: 0.4843
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9160 - loss: 0.4631 - val_acc: 0.8421 - val_loss: 0.4132
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9187 - loss: 0.3989 - val_acc: 0.9474 - val_loss: 0.3467
Epoch 9/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.9760 - loss: 0.3154 - v